# Long-Running Memory Demo (UserInfoMemory Feature)

This notebook demonstrates the **UserInfoMemory** feature - a long-running memory system that extracts and remembers user information (name, persona) across conversations.

## What You'll Learn
- How to enable/disable memory with the `enable_memory` flag
- How UserInfoMemory extracts name and persona from user messages
- How agents use remembered information to personalize responses
- How memory persists across messages within a session
- Difference between memory-enabled and memory-disabled conversations
- Best practices for using long-running memory

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

## Memory Feature Overview
- **Memory OFF (default)**: Agent responds generically, doesn't extract or remember user info
- **Memory ON**: Agent extracts name/persona and personalizes future responses

## Setup - Install Required Packages

In [ ]:
#r "nuget: System.Net.Http.Json"
#r "nuget: System.Text.Json"

using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Json;
using System.Text.Json.Serialization;

## Configure API Connection

In [ ]:
// API Configuration
var apiBaseUrl = "http://localhost:8000";
var httpClient = new HttpClient { BaseAddress = new Uri(apiBaseUrl) };
httpClient.Timeout = TimeSpan.FromMinutes(2);

Console.WriteLine($"✅ Connected to API: {apiBaseUrl}");
Console.WriteLine($"🧠 UserInfoMemory Demo\n");

## Scenario 1: Memory DISABLED (Default Behavior)

Without enabling memory, the agent responds generically and doesn't extract user information.

In [ ]:
Console.WriteLine("💭 Scenario 1: Memory DISABLED");
Console.WriteLine("================================\n");

// First message - introduce yourself
var messageNoMemory1 = new
{
    message = "Hi! My name is Alice and I'm a software engineer.",
    agents = new[] { "azure_openai_agent" },
    enable_memory = false  // Memory disabled
};

Console.WriteLine($"[Turn 1] 👤 User: {messageNoMemory1.message}");
Console.WriteLine($"         Memory: {(messageNoMemory1.enable_memory ? "ON 🧠" : "OFF 💭")}\n");

var responseNoMem1 = await httpClient.PostAsJsonAsync("/chat", messageNoMemory1);
var resultNoMem1 = await responseNoMem1.Content.ReadFromJsonAsync<JsonElement>();
var sessionNoMemory = resultNoMem1.GetProperty("session_id").GetString();

Console.WriteLine($"🤖 Agent: {resultNoMem1.GetProperty("content").GetString()}");
Console.WriteLine($"\n📊 Session ID: {sessionNoMemory}");

await Task.Delay(1000);

// Follow-up question
var messageNoMemory2 = new
{
    message = "What's my name and what do I do?",
    session_id = sessionNoMemory,
    agents = new[] { "azure_openai_agent" },
    enable_memory = false
};

Console.WriteLine($"\n[Turn 2] 👤 User: {messageNoMemory2.message}");
Console.WriteLine($"         Memory: {(messageNoMemory2.enable_memory ? "ON 🧠" : "OFF 💭")}\n");

var responseNoMem2 = await httpClient.PostAsJsonAsync("/chat", messageNoMemory2);
var resultNoMem2 = await responseNoMem2.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine($"🤖 Agent: {resultNoMem2.GetProperty("content").GetString()}");
Console.WriteLine($"\n⚠️ Note: Agent doesn't remember your name because memory was disabled!");

## Scenario 2: Memory ENABLED

With memory enabled, the agent extracts and remembers your name and persona.

In [ ]:
Console.WriteLine("\n\n🧠 Scenario 2: Memory ENABLED");
Console.WriteLine("================================\n");

// First message - introduce yourself with memory enabled
var messageWithMemory1 = new
{
    message = "Hi! My name is Bob and I'm a data scientist.",
    agents = new[] { "azure_openai_agent" },
    enable_memory = true  // Memory enabled!
};

Console.WriteLine($"[Turn 1] 👤 User: {messageWithMemory1.message}");
Console.WriteLine($"         Memory: {(messageWithMemory1.enable_memory ? "ON 🧠" : "OFF 💭")}\n");

var responseWithMem1 = await httpClient.PostAsJsonAsync("/chat", messageWithMemory1);
var resultWithMem1 = await responseWithMem1.Content.ReadFromJsonAsync<JsonElement>();
var sessionWithMemory = resultWithMem1.GetProperty("session_id").GetString();

Console.WriteLine($"🤖 Agent: {resultWithMem1.GetProperty("content").GetString()}");
Console.WriteLine($"\n📊 Session ID: {sessionWithMemory}");
Console.WriteLine($"✅ Memory extracted: Name='Bob', Persona='data scientist'");

await Task.Delay(1000);

// Follow-up question
var messageWithMemory2 = new
{
    message = "What's my name and what do I do?",
    session_id = sessionWithMemory,
    agents = new[] { "azure_openai_agent" },
    enable_memory = true
};

Console.WriteLine($"\n[Turn 2] 👤 User: {messageWithMemory2.message}");
Console.WriteLine($"         Memory: {(messageWithMemory2.enable_memory ? "ON 🧠" : "OFF 💭")}\n");

var responseWithMem2 = await httpClient.PostAsJsonAsync("/chat", messageWithMemory2);
var resultWithMem2 = await responseWithMem2.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine($"🤖 Agent: {resultWithMem2.GetProperty("content").GetString()}");
Console.WriteLine($"\n✅ Success! Agent remembered your name and role!");

## Scenario 3: Multi-Turn Personalized Conversation

See how memory enables personalized, context-aware responses across multiple turns.

In [ ]:
Console.WriteLine("\n\n🎭 Scenario 3: Personalized Multi-Turn Conversation");
Console.WriteLine("====================================================\n");

// Start new session with detailed introduction
var personalizedMsg1 = new
{
    message = "Hello! I'm Sarah, a marketing manager at TechCorp, and I'm 28 years old.",
    agents = new[] { "azure_openai_agent" },
    enable_memory = true
};

Console.WriteLine($"[Turn 1] 👤 User: {personalizedMsg1.message}");
Console.WriteLine($"         Memory: ON 🧠\n");

var personalResp1 = await httpClient.PostAsJsonAsync("/chat", personalizedMsg1);
var personalResult1 = await personalResp1.Content.ReadFromJsonAsync<JsonElement>();
var personalSession = personalResult1.GetProperty("session_id").GetString();

Console.WriteLine($"🤖 Agent: {personalResult1.GetProperty("content").GetString()}");
Console.WriteLine($"\n✅ Extracted: Name='Sarah', Persona='marketing manager at TechCorp, 28 years old'");

// Series of follow-up questions
var followUpQuestions = new[]
{
    "Can you remind me what I do for work?",
    "What's my name again?",
    "Summarize what you know about me.",
    "Give me some career advice based on my background."
};

int turnNum = 2;
foreach (var question in followUpQuestions)
{
    await Task.Delay(1000);
    
    Console.WriteLine($"\n[Turn {turnNum}] 👤 User: {question}");
    Console.WriteLine($"           Memory: ON 🧠\n");
    
    var followUpMsg = new
    {
        message = question,
        session_id = personalSession,
        agents = new[] { "azure_openai_agent" },
        enable_memory = true
    };
    
    var followUpResp = await httpClient.PostAsJsonAsync("/chat", followUpMsg);
    var followUpResult = await followUpResp.Content.ReadFromJsonAsync<JsonElement>();
    
    Console.WriteLine($"🤖 Agent: {followUpResult.GetProperty("content").GetString()}");
    
    turnNum++;
}

Console.WriteLine($"\n\n✅ All responses were personalized using remembered information!");

## Scenario 4: Memory with Multiple Agents (Group Chat)

UserInfoMemory works seamlessly in multi-agent scenarios.

In [ ]:
Console.WriteLine("\n\n👥 Scenario 4: Memory with Multiple Agents");
Console.WriteLine("============================================\n");

// First message with multiple agents
var multiAgentMemMsg1 = new
{
    message = "Hi! I'm Michael, a cloud architect specializing in Azure. I need help planning a migration.",
    agents = new[] { "azure_openai_agent", "knowledge_finder" },
    max_turns = 3,
    enable_memory = true,
    format = "user_friendly"
};

Console.WriteLine($"[Turn 1] 👤 User: {multiAgentMemMsg1.message}");
Console.WriteLine($"         Memory: ON 🧠");
Console.WriteLine($"         Agents: {string.Join(", ", multiAgentMemMsg1.agents)}\n");

var multiMemResp1 = await httpClient.PostAsJsonAsync("/chat", multiAgentMemMsg1);
var multiMemResult1 = await multiMemResp1.Content.ReadFromJsonAsync<JsonElement>();
var multiMemSession = multiMemResult1.GetProperty("session_id").GetString();

var content1 = multiMemResult1.GetProperty("content").GetString();
Console.WriteLine($"🤖 Response: {(content1.Length > 300 ? content1.Substring(0, 300) + "..." : content1)}");
Console.WriteLine($"\n✅ Memory extracted: Name='Michael', Persona='cloud architect specializing in Azure'");

await Task.Delay(1500);

// Follow-up leveraging memory
var multiAgentMemMsg2 = new
{
    message = "What expertise do I have that would help with this migration?",
    session_id = multiMemSession,
    agents = new[] { "azure_openai_agent", "knowledge_finder" },
    max_turns = 3,
    enable_memory = true,
    format = "user_friendly"
};

Console.WriteLine($"\n[Turn 2] 👤 User: {multiAgentMemMsg2.message}");
Console.WriteLine($"         Memory: ON 🧠\n");

var multiMemResp2 = await httpClient.PostAsJsonAsync("/chat", multiAgentMemMsg2);
var multiMemResult2 = await multiMemResp2.Content.ReadFromJsonAsync<JsonElement>();

var content2 = multiMemResult2.GetProperty("content").GetString();
Console.WriteLine($"🤖 Response: {(content2.Length > 300 ? content2.Substring(0, 300) + "..." : content2)}");

// Show metadata
if (multiMemResult2.TryGetProperty("metadata", out var multiMemMeta))
{
    Console.WriteLine($"\n📊 Collaboration Stats:");
    Console.WriteLine($"   Agent Count: {multiMemMeta.GetProperty("agent_count").GetInt32()}");
    
    if (multiMemMeta.TryGetProperty("contributing_agents", out var agents))
    {
        var agentList = string.Join(", ", agents.EnumerateArray().Select(a => a.GetString()));
        Console.WriteLine($"   Agents Used: {agentList}");
    }
}

Console.WriteLine($"\n✅ All agents in the group chat have access to the same memory!");

## Scenario 5: Testing Information Extraction Patterns

See how UserInfoMemory extracts different types of information.

In [ ]:
Console.WriteLine("\n\n🔍 Scenario 5: Information Extraction Patterns");
Console.WriteLine("================================================\n");

var extractionTests = new[]
{
    new { Input = "My name is Jennifer", Expected = "Name: Jennifer" },
    new { Input = "I'm a developer", Expected = "Persona: developer" },
    new { Input = "Call me Mike and I work as a project manager", Expected = "Name: Mike, Persona: project manager" },
    new { Input = "I'm 35 and I'm a teacher", Expected = "Persona: 35 years old, teacher" }
};

int testNum = 1;
foreach (var test in extractionTests)
{
    Console.WriteLine($"\n[Test {testNum}]");
    Console.WriteLine($"Input: \"{test.Input}\"");
    Console.WriteLine($"Expected: {test.Expected}\n");
    
    var extractMsg = new
    {
        message = test.Input,
        agents = new[] { "azure_openai_agent" },
        enable_memory = true
    };
    
    var extractResp = await httpClient.PostAsJsonAsync("/chat", extractMsg);
    var extractResult = await extractResp.Content.ReadFromJsonAsync<JsonElement>();
    
    Console.WriteLine($"🤖 Agent Response: {extractResult.GetProperty("content").GetString()}");
    Console.WriteLine($"✅ Information extracted and stored in session");
    
    testNum++;
    await Task.Delay(800);
}

Console.WriteLine($"\n\n✅ UserInfoMemory successfully extracts various information patterns!");

## Best Practices for Using Memory

In [ ]:
Console.WriteLine("📚 UserInfoMemory Best Practices");
Console.WriteLine("=================================\n");

Console.WriteLine("✅ DO:");
Console.WriteLine("   1. Enable memory for personalized, multi-turn conversations");
Console.WriteLine("   2. Let users introduce themselves naturally");
Console.WriteLine("   3. Use memory in customer service scenarios");
Console.WriteLine("   4. Combine memory with session management for best results");
Console.WriteLine("   5. Provide a UI toggle to let users control memory");
Console.WriteLine();

Console.WriteLine("❌ DON'T:");
Console.WriteLine("   1. Enable memory for anonymous/privacy-sensitive conversations");
Console.WriteLine("   2. Store sensitive personal information via memory");
Console.WriteLine("   3. Expect memory to work without a session_id");
Console.WriteLine("   4. Use memory for one-off queries");
Console.WriteLine();

Console.WriteLine("💡 Tips:");
Console.WriteLine("   • Memory extracts: name, persona/role, age");
Console.WriteLine("   • Works best with explicit information (\"My name is...\", \"I'm a...\") ");
Console.WriteLine("   • Memory persists for the entire session");
Console.WriteLine("   • Can be enabled per-request with 'enable_memory' flag");
Console.WriteLine("   • Falls back gracefully if extraction fails");
Console.WriteLine();

Console.WriteLine("🎯 When to Use Memory:");
Console.WriteLine("   ✅ Customer support conversations");
Console.WriteLine("   ✅ Personal assistant scenarios");
Console.WriteLine("   ✅ Tutoring or coaching interactions");
Console.WriteLine("   ✅ Long-running task assistance");
Console.WriteLine();

Console.WriteLine("🚫 When NOT to Use Memory:");
Console.WriteLine("   ❌ Anonymous information queries");
Console.WriteLine("   ❌ Privacy-sensitive conversations");
Console.WriteLine("   ❌ Quick one-time questions");
Console.WriteLine("   ❌ Public/shared agent instances");

## Memory Configuration and Control

In [ ]:
Console.WriteLine("⚙️ Memory Configuration");
Console.WriteLine("========================\n");

Console.WriteLine("🎚️ Control Methods:");
Console.WriteLine();

Console.WriteLine("1. Request-level (Recommended):");
Console.WriteLine("   POST /chat");
Console.WriteLine("   {");
Console.WriteLine("     \"message\": \"Your message\",");
Console.WriteLine("     \"enable_memory\": true    ← Control per request");
Console.WriteLine("   }");
Console.WriteLine();

Console.WriteLine("2. Environment Variable (Default):");
Console.WriteLine("   File: .env");
Console.WriteLine("   ENABLE_LONG_RUNNING_MEMORY=\"true\"");
Console.WriteLine();

Console.WriteLine("3. UI Toggle (Frontend):");
Console.WriteLine("   • React: <MemoryToggle> component");
Console.WriteLine("   • HTML: memory-demo.html");
Console.WriteLine("   • Main App: Sidebar toggle in App.js");
Console.WriteLine();

Console.WriteLine("🔄 Priority Order:");
Console.WriteLine("   1. Request Flag (enable_memory) - Highest priority");
Console.WriteLine("   2. Environment Variable (ENABLE_LONG_RUNNING_MEMORY)");
Console.WriteLine("   3. Default (false) - Lowest priority");
Console.WriteLine();

Console.WriteLine("📊 Memory State:");
Console.WriteLine("   • Stored in: Session state (UserInfoMemory provider)");
Console.WriteLine("   • Lifetime: Duration of session");
Console.WriteLine("   • Scope: Per-session (isolated between sessions)");
Console.WriteLine("   • Persistence: In-memory (not persisted to disk)");
Console.WriteLine();

Console.WriteLine("🔍 What Gets Stored:");
Console.WriteLine("   • UserName: Extracted from name patterns");
Console.WriteLine("   • UserPersona: Role, occupation, age");
Console.WriteLine("   • HasAskedForName: Internal tracking flag");
Console.WriteLine("   • HasAskedForPersona: Internal tracking flag");

## Summary

In this notebook, you learned:
- ✅ How to enable/disable UserInfoMemory with `enable_memory` flag
- ✅ How memory extracts name and persona from user messages
- ✅ Difference between memory-enabled and disabled conversations
- ✅ How memory enhances personalization across multiple turns
- ✅ Memory works seamlessly with multi-agent scenarios
- ✅ Various information extraction patterns
- ✅ Best practices and when to use memory

## Key Features

### UserInfoMemory
- Automatically extracts user name and persona from messages
- Stores information in session state
- Personalizes agent responses using remembered info
- Works with both single and multi-agent conversations

### Control & Configuration
- Per-request control via `enable_memory` flag
- Environment variable for default behavior
- UI toggles in frontend applications
- Priority: Request > Environment > Default

### Extraction Patterns
- **Name**: "My name is X", "I'm X", "Call me X"
- **Persona**: "I'm a developer", "I work as X", "I'm 25"
- Falls back gracefully if extraction fails

## Real-World Use Cases
1. **Customer Support**: Remember customer details across conversation
2. **Personal Assistants**: Personalize suggestions based on user role
3. **Coaching/Tutoring**: Adapt teaching style to student background
4. **Sales**: Remember customer preferences and needs

## UI Integration
- **Main App**: Memory toggle in sidebar (App.js)
- **HTML Demo**: memory-demo.html with live stats
- **React Component**: MemoryToggle.jsx (reusable)
- **Debug Tool**: memory-test.html for testing

## Next Steps
- Try the frontend demos to see memory in action
- Review the **Single Agent Demo** for basic agent interaction
- Explore the **Multiple Agents Demo** for group chat scenarios
- Check the **Content Safety Demo** for content moderation